In [1]:
%matplotlib inline

In [1]:
cd ..

C:\Projects\python\recommender


In [2]:
from functools import partial
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import torch as T
import torch.optim as optim

from utils import get_log_dir

In [3]:
from datasets import SeqStackoverflow
from models import FMLearner, TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM, TransProbFM
from models.fm_learner import simple_loss, trans_loss, simple_weight_loss, trans_weight_loss

In [4]:
DEVICE = T.cuda.current_device()
BATCH = 1
SHUFFLE = True
WORKERS = 0
NEG_SAMPLE = 5
item_path = Path("./inputs/stackoverflow/item.csv")

In [5]:
db = SeqStackoverflow(data_path=item_path, min_user=4)
db

Original user size: 75572
Filtered user size: 14795
Original item dataframe shape: (299845, 9)
Filtered item dataframe shape: (217223, 9)
Remove number of questions less than 10 date
Filter item dateframe shape: (217146, 9)


In [6]:
db.config_db(batch_size=BATCH,
             shuffle=SHUFFLE,
             num_workers=WORKERS,
             device=DEVICE,
             neg_sample=NEG_SAMPLE)

## Create Criterion

In [7]:
# regst setting
LINEAR_REG = 1
EMB_REG = 1
TRANS_REG = 1

In [8]:
simple_loss_callback = partial(simple_loss, LINEAR_REG, EMB_REG)
simple_loss_callback

functools.partial(<function simple_loss at 0x0000025A893E1C18>, 1, 1)

In [9]:
trans_loss_callback = partial(trans_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_loss_callback

functools.partial(<function trans_loss at 0x0000025A89433318>, 1, 1, 1)

In [10]:
simple_weight_loss_callback = partial(simple_weight_loss, LINEAR_REG, EMB_REG)
simple_weight_loss_callback

functools.partial(<function simple_weight_loss at 0x0000025A894333A8>, 1, 1)

In [11]:
trans_weight_loss_callback = partial(trans_weight_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_weight_loss_callback

functools.partial(<function trans_weight_loss at 0x0000025A89433438>, 1, 1, 1)

## Train model

### Hyper-parameter

In [12]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1
INIT_SCALE = 0.001

## Train fm model

In [13]:
LEARNING_RATE = 0.001
DECAY_FREQ = 1000
DECAY_GAMMA = 1

In [14]:
fm_model = TorchFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
fm_model

TorchFM()

In [23]:
adam_opt = optim.Adam(fm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [24]:
fm_learner = FMLearner(fm_model, adam_opt, schedular, db)
fm_learner

In [18]:
fm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [19]:
fm_learner.compile(train_col='seq',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [25]:
fm_learner.compile(train_col='seq',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_weight_loss_callback)

In [20]:
fm_learner.fit(epoch=3,
               log_dir=get_log_dir(ds_type='simple_stackoverflow',
                                   model_type='fm'))

  0%|                                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 165667.38797941408
Epoch 0 step 1: training accuarcy: 0.3044
Epoch 0 step 1: training loss: 160522.81291822562
Epoch 0 step 2: training accuarcy: 0.33480000000000004
Epoch 0 step 2: training loss: 155331.54568408176
Epoch 0 step 3: training accuarcy: 0.3816
Epoch 0 step 3: training loss: 150645.502428096
Epoch 0 step 4: training accuarcy: 0.4128
Epoch 0 step 4: training loss: 145525.92265924456
Epoch 0 step 5: training accuarcy: 0.4803
Epoch 0 step 5: training loss: 141141.7625060359
Epoch 0 step 6: training accuarcy: 0.5015000000000001
Epoch 0 step 6: training loss: 136641.41357274924
Epoch 0 step 7: training accuarcy: 0.5422
Epoch 0 step 7: training loss: 132610.45741318355
Epoch 0 step 8: training accuarcy: 0.5813
Epoch 0 step 8: training loss: 128733.55214231642
Epoch 0 step 9: training accuarcy: 0.5937
Epoch 0 step 9: training loss: 124958.7235743972
Epoch 0 step 10: training accuarcy: 0.6092000000000001
Epoch 0 step 10: training loss: 12080

 33%|█████████████████████████████████████████████████████████▎                                                                                                                  | 1/3 [05:58<11:56, 358.40s/it]

Epoch: 1
Epoch 1 step 94: training loss: 17253.000294836093
Epoch 1 step 95: training accuarcy: 0.9386
Epoch 1 step 95: training loss: 16929.44694080417
Epoch 1 step 96: training accuarcy: 0.9470000000000001
Epoch 1 step 96: training loss: 16618.178810533795
Epoch 1 step 97: training accuarcy: 0.9502
Epoch 1 step 97: training loss: 16417.661590538908
Epoch 1 step 98: training accuarcy: 0.9473
Epoch 1 step 98: training loss: 16143.834062044281
Epoch 1 step 99: training accuarcy: 0.9475
Epoch 1 step 99: training loss: 15896.39660381337
Epoch 1 step 100: training accuarcy: 0.9490000000000001
Epoch 1 step 100: training loss: 15748.38926558424
Epoch 1 step 101: training accuarcy: 0.9419000000000001
Epoch 1 step 101: training loss: 15386.377592670276
Epoch 1 step 102: training accuarcy: 0.9540000000000001
Epoch 1 step 102: training loss: 15376.393116589657
Epoch 1 step 103: training accuarcy: 0.9312
Epoch 1 step 103: training loss: 15011.044883610624
Epoch 1 step 104: training accuarcy: 0.94

 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                         | 2/3 [11:51<05:56, 356.91s/it]

Epoch: 2
Epoch 2 step 188: training loss: 5207.265124418872
Epoch 2 step 189: training accuarcy: 0.9912000000000001
Epoch 2 step 189: training loss: 5137.740812057923
Epoch 2 step 190: training accuarcy: 0.9938
Epoch 2 step 190: training loss: 5085.0555033591545
Epoch 2 step 191: training accuarcy: 0.9931000000000001
Epoch 2 step 191: training loss: 5022.475002910604
Epoch 2 step 192: training accuarcy: 0.9939
Epoch 2 step 192: training loss: 4962.457580573859
Epoch 2 step 193: training accuarcy: 0.9942000000000001
Epoch 2 step 193: training loss: 4918.621780876371
Epoch 2 step 194: training accuarcy: 0.993
Epoch 2 step 194: training loss: 4849.670847300937
Epoch 2 step 195: training accuarcy: 0.9955
Epoch 2 step 195: training loss: 4823.15943835334
Epoch 2 step 196: training accuarcy: 0.9933000000000001
Epoch 2 step 196: training loss: 4763.021790635826
Epoch 2 step 197: training accuarcy: 0.9929
Epoch 2 step 197: training loss: 4689.585151847931
Epoch 2 step 198: training accuarcy: 0

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [17:47<00:00, 356.55s/it]


In [26]:
fm_learner.fit(epoch=3,
               log_dir=get_log_dir(ds_type='weight_stackoverflow',
                                   model_type='fm'))

  0%|                                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 201267.01323947922
Epoch 0 step 1: training accuarcy: 0.2868
Epoch 0 step 1: training loss: 192904.08661832023
Epoch 0 step 2: training accuarcy: 0.34790000000000004
Epoch 0 step 2: training loss: 185050.9434924109
Epoch 0 step 3: training accuarcy: 0.39890000000000003
Epoch 0 step 3: training loss: 177390.06584664635
Epoch 0 step 4: training accuarcy: 0.4494
Epoch 0 step 4: training loss: 170233.2167723767
Epoch 0 step 5: training accuarcy: 0.5005000000000001
Epoch 0 step 5: training loss: 161337.98932542282
Epoch 0 step 6: training accuarcy: 0.5667
Epoch 0 step 6: training loss: 154429.88840470806
Epoch 0 step 7: training accuarcy: 0.6111
Epoch 0 step 7: training loss: 147550.57053682234
Epoch 0 step 8: training accuarcy: 0.6662
Epoch 0 step 8: training loss: 142155.28672222613
Epoch 0 step 9: training accuarcy: 0.6998000000000001
Epoch 0 step 9: training loss: 136540.6399461447
Epoch 0 step 10: training accuarcy: 0.7302000000000001
Epoch 0 ste

 33%|█████████████████████████████████████████████████████████▎                                                                                                                  | 1/3 [05:59<11:59, 359.68s/it]

Epoch: 1
Epoch 1 step 94: training loss: 21177.200945431392
Epoch 1 step 95: training accuarcy: 0.9995
Epoch 1 step 95: training loss: 20979.134364941914
Epoch 1 step 96: training accuarcy: 0.9996
Epoch 1 step 96: training loss: 20804.850597347526
Epoch 1 step 97: training accuarcy: 0.9989
Epoch 1 step 97: training loss: 20594.936166317028
Epoch 1 step 98: training accuarcy: 0.9988
Epoch 1 step 98: training loss: 20376.69277913169
Epoch 1 step 99: training accuarcy: 0.9997
Epoch 1 step 99: training loss: 20161.61100274686
Epoch 1 step 100: training accuarcy: 0.9991000000000001
Epoch 1 step 100: training loss: 20025.544135266195
Epoch 1 step 101: training accuarcy: 0.9985
Epoch 1 step 101: training loss: 19865.46342637483
Epoch 1 step 102: training accuarcy: 0.998
Epoch 1 step 102: training loss: 19723.00470830216
Epoch 1 step 103: training accuarcy: 0.9981000000000001
Epoch 1 step 103: training loss: 19523.435445045267
Epoch 1 step 104: training accuarcy: 0.9982000000000001
Epoch 1 ste

Epoch 1 step 178: training loss: 12802.40172813432
Epoch 1 step 179: training accuarcy: 1.0
Epoch 1 step 179: training loss: 12745.240163237848
Epoch 1 step 180: training accuarcy: 1.0
Epoch 1 step 180: training loss: 12695.210112532448
Epoch 1 step 181: training accuarcy: 0.9999
Epoch 1 step 181: training loss: 12698.079433262585
Epoch 1 step 182: training accuarcy: 0.9985
Epoch 1 step 182: training loss: 12600.41668554977
Epoch 1 step 183: training accuarcy: 0.9996
Epoch 1 step 183: training loss: 12565.675429981968
Epoch 1 step 184: training accuarcy: 0.9995
Epoch 1 step 184: training loss: 12477.439825444853
Epoch 1 step 185: training accuarcy: 1.0
Epoch 1 step 185: training loss: 12402.94862374809
Epoch 1 step 186: training accuarcy: 1.0
Epoch 1 step 186: training loss: 12382.622871603588
Epoch 1 step 187: training accuarcy: 1.0
Epoch 1 step 187: training loss: 12308.77604860306
Epoch 1 step 188: training accuarcy: 1.0
Epoch 1: train loss 15732.679137420417, train accuarcy 0.99903

 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                         | 2/3 [12:09<06:02, 362.71s/it]

Epoch: 2
Epoch 2 step 188: training loss: 12300.236456950415
Epoch 2 step 189: training accuarcy: 0.9993000000000001
Epoch 2 step 189: training loss: 12200.950215147088
Epoch 2 step 190: training accuarcy: 1.0
Epoch 2 step 190: training loss: 12167.176559371544
Epoch 2 step 191: training accuarcy: 0.9995
Epoch 2 step 191: training loss: 12101.416119692034
Epoch 2 step 192: training accuarcy: 1.0
Epoch 2 step 192: training loss: 12039.16645057354
Epoch 2 step 193: training accuarcy: 1.0
Epoch 2 step 193: training loss: 11989.235659360593
Epoch 2 step 194: training accuarcy: 1.0
Epoch 2 step 194: training loss: 11927.153164028949
Epoch 2 step 195: training accuarcy: 1.0
Epoch 2 step 195: training loss: 11874.240504642654
Epoch 2 step 196: training accuarcy: 1.0
Epoch 2 step 196: training loss: 11824.391694250335
Epoch 2 step 197: training accuarcy: 1.0
Epoch 2 step 197: training loss: 11796.360362880008
Epoch 2 step 198: training accuarcy: 0.9995
Epoch 2 step 198: training loss: 11720.97

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [18:09<00:00, 361.97s/it]


In [27]:
del fm_model
T.cuda.empty_cache()

### Train HRM FM Model

In [15]:
hrm_model = TorchHrmFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
hrm_model

TorchHrmFM()

In [16]:
adam_opt = optim.Adam(hrm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [17]:
hrm_learner = FMLearner(hrm_model, adam_opt, schedular, db)
hrm_learner

In [18]:
hrm_learner.compile(train_col='base',
                    valid_col='base',
                    test_col='base',
                    loss_callback=simple_loss_callback)

In [18]:
hrm_learner.compile(train_col='seq',
                    valid_col='seq',
                    test_col='seq',
                    loss_callback=simple_loss_callback)

In [24]:
hrm_learner.compile(train_col='seq',
                    valid_col='seq',
                    test_col='seq',
                    loss_callback=simple_weight_loss_callback)

In [19]:
hrm_learner.fit(epoch=3,
                log_dir=get_log_dir('stackoverflow', 'hrm'))

  0%|                                                                                                                                                           | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 159048.11073367047
Epoch 0 step 1: training accuarcy: 0.505
Epoch 0 step 1: training loss: 154506.19935959077
Epoch 0 step 2: training accuarcy: 0.5005000000000001
Epoch 0 step 2: training loss: 149923.50724870843
Epoch 0 step 3: training accuarcy: 0.515
Epoch 0 step 3: training loss: 145353.72562254092
Epoch 0 step 4: training accuarcy: 0.5125
Epoch 0 step 4: training loss: 141115.16804273357
Epoch 0 step 5: training accuarcy: 0.525
Epoch 0 step 5: training loss: 137020.64433766948
Epoch 0 step 6: training accuarcy: 0.5045000000000001
Epoch 0 step 6: training loss: 132754.3621155682
Epoch 0 step 7: training accuarcy: 0.516
Epoch 0 step 7: training loss: 128978.30534832862
Epoch 0 step 8: training accuarcy: 0.5135
Epoch 0 step 8: training loss: 124930.95152145148
Epoch 0 step 9: training accuarcy: 0.506
Epoch 0 step 9: training loss: 121251.83028566878
Epoch 0 step 10: training accuarcy: 0.4815
Epoch 0 step 10: training loss: 117342.76410652416
E

 33%|█████████████████████████████████████████████████                                                                                                  | 1/3 [00:56<01:53, 56.95s/it]

Epoch: 1
Epoch 1 step 94: training loss: 6014.63140258411
Epoch 1 step 95: training accuarcy: 0.589
Epoch 1 step 95: training loss: 5835.817168060788
Epoch 1 step 96: training accuarcy: 0.5935
Epoch 1 step 96: training loss: 5637.818024932507
Epoch 1 step 97: training accuarcy: 0.6095
Epoch 1 step 97: training loss: 5477.173925267155
Epoch 1 step 98: training accuarcy: 0.5795
Epoch 1 step 98: training loss: 5267.386195129975
Epoch 1 step 99: training accuarcy: 0.5915
Epoch 1 step 99: training loss: 5132.098627529942
Epoch 1 step 100: training accuarcy: 0.5785
Epoch 1 step 100: training loss: 4946.128995229653
Epoch 1 step 101: training accuarcy: 0.5995
Epoch 1 step 101: training loss: 4770.134827383252
Epoch 1 step 102: training accuarcy: 0.6025
Epoch 1 step 102: training loss: 4631.962320699361
Epoch 1 step 103: training accuarcy: 0.5925
Epoch 1 step 103: training loss: 4473.191373237095
Epoch 1 step 104: training accuarcy: 0.61
Epoch 1 step 104: training loss: 4378.759143034718
Epoch

 67%|██████████████████████████████████████████████████████████████████████████████████████████████████                                                 | 2/3 [01:56<00:57, 57.77s/it]

Epoch: 2
Epoch 2 step 188: training loss: 1343.352135967653
Epoch 2 step 189: training accuarcy: 0.6635
Epoch 2 step 189: training loss: 1351.8539191791665
Epoch 2 step 190: training accuarcy: 0.6415
Epoch 2 step 190: training loss: 1336.162051250011
Epoch 2 step 191: training accuarcy: 0.661
Epoch 2 step 191: training loss: 1326.1843900077347
Epoch 2 step 192: training accuarcy: 0.671
Epoch 2 step 192: training loss: 1329.3462069128368
Epoch 2 step 193: training accuarcy: 0.6665
Epoch 2 step 193: training loss: 1334.9808278057824
Epoch 2 step 194: training accuarcy: 0.6535
Epoch 2 step 194: training loss: 1330.6799917493702
Epoch 2 step 195: training accuarcy: 0.661
Epoch 2 step 195: training loss: 1336.767393938974
Epoch 2 step 196: training accuarcy: 0.657
Epoch 2 step 196: training loss: 1325.717265675164
Epoch 2 step 197: training accuarcy: 0.659
Epoch 2 step 197: training loss: 1321.0124780255508
Epoch 2 step 198: training accuarcy: 0.6725
Epoch 2 step 198: training loss: 1330.00

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:52<00:00, 57.55s/it]


In [25]:
hrm_learner.fit(epoch=3,
                log_dir=get_log_dir('weight_stackoverflow', 'hrm'))

  0%|                                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 254538.6787255127
Epoch 0 step 1: training accuarcy: 0.513
Epoch 0 step 1: training loss: 251949.33228869695
Epoch 0 step 2: training accuarcy: 0.5004000000000001
Epoch 0 step 2: training loss: 241641.8210724932
Epoch 0 step 3: training accuarcy: 0.5118
Epoch 0 step 3: training loss: 239113.87387257142
Epoch 0 step 4: training accuarcy: 0.5017
Epoch 0 step 4: training loss: 230531.48974914628
Epoch 0 step 5: training accuarcy: 0.5063
Epoch 0 step 5: training loss: 223105.2858104313
Epoch 0 step 6: training accuarcy: 0.5175000000000001
Epoch 0 step 6: training loss: 219010.16818752285
Epoch 0 step 7: training accuarcy: 0.5139
Epoch 0 step 7: training loss: 214528.18459561194
Epoch 0 step 8: training accuarcy: 0.5054000000000001
Epoch 0 step 8: training loss: 208287.07559957518
Epoch 0 step 9: training accuarcy: 0.5082
Epoch 0 step 9: training loss: 206418.83050568268
Epoch 0 step 10: training accuarcy: 0.5066
Epoch 0 step 10: training loss: 201511

 33%|█████████████████████████████████████████████████████████▎                                                                                                                  | 1/3 [06:04<12:08, 364.43s/it]

Epoch: 1
Epoch 1 step 94: training loss: 56659.12969430812
Epoch 1 step 95: training accuarcy: 0.5716
Epoch 1 step 95: training loss: 57376.00750637949
Epoch 1 step 96: training accuarcy: 0.5619000000000001
Epoch 1 step 96: training loss: 56754.7262686348
Epoch 1 step 97: training accuarcy: 0.5655
Epoch 1 step 97: training loss: 57165.47242273083
Epoch 1 step 98: training accuarcy: 0.5468000000000001
Epoch 1 step 98: training loss: 56356.28627938623
Epoch 1 step 99: training accuarcy: 0.5693
Epoch 1 step 99: training loss: 55585.787985306066
Epoch 1 step 100: training accuarcy: 0.5705
Epoch 1 step 100: training loss: 55499.549072200825
Epoch 1 step 101: training accuarcy: 0.5690000000000001
Epoch 1 step 101: training loss: 54683.249371493366
Epoch 1 step 102: training accuarcy: 0.5697
Epoch 1 step 102: training loss: 54889.673526328144
Epoch 1 step 103: training accuarcy: 0.5618000000000001
Epoch 1 step 103: training loss: 54782.5382839802
Epoch 1 step 104: training accuarcy: 0.5614
Ep

 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                         | 2/3 [12:00<06:01, 361.83s/it]

Epoch: 2
Epoch 2 step 188: training loss: 38289.70383362433
Epoch 2 step 189: training accuarcy: 0.6124
Epoch 2 step 189: training loss: 37846.75054984278
Epoch 2 step 190: training accuarcy: 0.6107
Epoch 2 step 190: training loss: 37324.85982707166
Epoch 2 step 191: training accuarcy: 0.6195
Epoch 2 step 191: training loss: 37533.978568402934
Epoch 2 step 192: training accuarcy: 0.6279
Epoch 2 step 192: training loss: 37929.87101757714
Epoch 2 step 193: training accuarcy: 0.6146
Epoch 2 step 193: training loss: 38065.29618775864
Epoch 2 step 194: training accuarcy: 0.6018
Epoch 2 step 194: training loss: 37259.72898786422
Epoch 2 step 195: training accuarcy: 0.6268
Epoch 2 step 195: training loss: 37633.22836372195
Epoch 2 step 196: training accuarcy: 0.6157
Epoch 2 step 196: training loss: 37158.84043409147
Epoch 2 step 197: training accuarcy: 0.6197
Epoch 2 step 197: training loss: 37415.47994092229
Epoch 2 step 198: training accuarcy: 0.619
Epoch 2 step 198: training loss: 37433.63

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [18:04<00:00, 362.42s/it]


In [20]:
del hrm_model
T.cuda.empty_cache()

### Train PRME FM Model

In [21]:
prme_model = TorchPrmeFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
prme_model

TorchPrmeFM()

In [22]:
adam_opt = optim.Adam(prme_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [23]:
prme_learner = FMLearner(prme_model, adam_opt, schedular, db)
prme_learner

In [24]:
prme_learner.compile(train_col='base',
                     valid_col='base',
                     test_col='base',
                     loss_callback=simple_loss_callback)

In [33]:
prme_learner.compile(train_col='seq',
                     valid_col='seq',
                     test_col='seq',
                     loss_callback=simple_loss_callback)

In [39]:
prme_learner.compile(train_col='seq',
                     valid_col='seq',
                     test_col='seq',
                     loss_callback=simple_weight_loss_callback)

In [25]:
prme_learner.fit(epoch=3, log_dir=get_log_dir('stackoverflow', 'prme'))

  0%|                                                                                                                                                           | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 163491.45589600064
Epoch 0 step 1: training accuarcy: 0.5115000000000001
Epoch 0 step 1: training loss: 158510.79386714342
Epoch 0 step 2: training accuarcy: 0.5145
Epoch 0 step 2: training loss: 154119.0812932362
Epoch 0 step 3: training accuarcy: 0.504
Epoch 0 step 3: training loss: 149246.3696957784
Epoch 0 step 4: training accuarcy: 0.5315
Epoch 0 step 4: training loss: 145427.39547512302
Epoch 0 step 5: training accuarcy: 0.507
Epoch 0 step 5: training loss: 141025.0067818816
Epoch 0 step 6: training accuarcy: 0.5
Epoch 0 step 6: training loss: 136510.11248979886
Epoch 0 step 7: training accuarcy: 0.5085000000000001
Epoch 0 step 7: training loss: 132654.4924816499
Epoch 0 step 8: training accuarcy: 0.509
Epoch 0 step 8: training loss: 129092.98993818159
Epoch 0 step 9: training accuarcy: 0.513
Epoch 0 step 9: training loss: 124770.45404526702
Epoch 0 step 10: training accuarcy: 0.5235
Epoch 0 step 10: training loss: 121220.48037450275
Epoch 

 33%|█████████████████████████████████████████████████                                                                                                  | 1/3 [00:56<01:53, 56.91s/it]

Epoch: 1
Epoch 1 step 94: training loss: 8293.994331702288
Epoch 1 step 95: training accuarcy: 0.5625
Epoch 1 step 95: training loss: 8064.755578936201
Epoch 1 step 96: training accuarcy: 0.5635
Epoch 1 step 96: training loss: 7842.842769963377
Epoch 1 step 97: training accuarcy: 0.556
Epoch 1 step 97: training loss: 7546.153042952386
Epoch 1 step 98: training accuarcy: 0.591
Epoch 1 step 98: training loss: 7387.078409653448
Epoch 1 step 99: training accuarcy: 0.557
Epoch 1 step 99: training loss: 7176.845219064235
Epoch 1 step 100: training accuarcy: 0.58
Epoch 1 step 100: training loss: 7019.935297055934
Epoch 1 step 101: training accuarcy: 0.5710000000000001
Epoch 1 step 101: training loss: 6740.0290087305275
Epoch 1 step 102: training accuarcy: 0.595
Epoch 1 step 102: training loss: 6550.224402164623
Epoch 1 step 103: training accuarcy: 0.578
Epoch 1 step 103: training loss: 6407.755371376894
Epoch 1 step 104: training accuarcy: 0.56
Epoch 1 step 104: training loss: 6260.9860112567

 67%|██████████████████████████████████████████████████████████████████████████████████████████████████                                                 | 2/3 [01:53<00:56, 56.77s/it]

Epoch: 2
Epoch 2 step 188: training loss: 1638.8651837236916
Epoch 2 step 189: training accuarcy: 0.6575
Epoch 2 step 189: training loss: 1588.0117323522081
Epoch 2 step 190: training accuarcy: 0.661
Epoch 2 step 190: training loss: 1617.4123395275878
Epoch 2 step 191: training accuarcy: 0.6405
Epoch 2 step 191: training loss: 1608.0608922709034
Epoch 2 step 192: training accuarcy: 0.6395000000000001
Epoch 2 step 192: training loss: 1624.2752138118435
Epoch 2 step 193: training accuarcy: 0.626
Epoch 2 step 193: training loss: 1577.5362230532435
Epoch 2 step 194: training accuarcy: 0.6575
Epoch 2 step 194: training loss: 1572.817225854195
Epoch 2 step 195: training accuarcy: 0.652
Epoch 2 step 195: training loss: 1552.3672621359467
Epoch 2 step 196: training accuarcy: 0.665
Epoch 2 step 196: training loss: 1560.2371217553373
Epoch 2 step 197: training accuarcy: 0.648
Epoch 2 step 197: training loss: 1530.740271487549
Epoch 2 step 198: training accuarcy: 0.6605
Epoch 2 step 198: training

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:51<00:00, 57.21s/it]


In [41]:
prme_learner.fit(epoch=5, log_dir=get_log_dir('weight_stackoverflow', 'prme'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 348012.7577500373
Epoch 0 step 1: training accuarcy: 0.5019
Epoch 0 step 1: training loss: 340914.66864264815
Epoch 0 step 2: training accuarcy: 0.5021
Epoch 0 step 2: training loss: 324089.08646529075
Epoch 0 step 3: training accuarcy: 0.5156000000000001
Epoch 0 step 3: training loss: 320449.7119097691
Epoch 0 step 4: training accuarcy: 0.5052
Epoch 0 step 4: training loss: 313406.75829188386
Epoch 0 step 5: training accuarcy: 0.503
Epoch 0 step 5: training loss: 310387.30414624815
Epoch 0 step 6: training accuarcy: 0.5047
Epoch 0 step 6: training loss: 308907.2514212637
Epoch 0 step 7: training accuarcy: 0.48710000000000003
Epoch 0 step 7: training loss: 297439.6338466797
Epoch 0 step 8: training accuarcy: 0.4914
Epoch 0 step 8: training loss: 289652.94739316235
Epoch 0 step 9: training accuarcy: 0.5075000000000001
Epoch 0 step 9: training loss: 278507.5054970049
Epoch 0 step 10: training accuarcy: 0.5044000000000001
Epoch 0 step 10: training l

 20%|██████████████████████████████████▍                                                                                                                                         | 1/5 [06:00<24:00, 360.16s/it]

Epoch: 1
Epoch 1 step 94: training loss: 77604.03784513936
Epoch 1 step 95: training accuarcy: 0.559
Epoch 1 step 95: training loss: 76699.43328647625
Epoch 1 step 96: training accuarcy: 0.5513
Epoch 1 step 96: training loss: 77558.49251401436
Epoch 1 step 97: training accuarcy: 0.5478000000000001
Epoch 1 step 97: training loss: 75428.98720322369
Epoch 1 step 98: training accuarcy: 0.5515
Epoch 1 step 98: training loss: 75071.65026798255
Epoch 1 step 99: training accuarcy: 0.5532
Epoch 1 step 99: training loss: 74816.88735191702
Epoch 1 step 100: training accuarcy: 0.5472
Epoch 1 step 100: training loss: 72816.99823899948
Epoch 1 step 101: training accuarcy: 0.5567
Epoch 1 step 101: training loss: 74478.70958747076
Epoch 1 step 102: training accuarcy: 0.5409
Epoch 1 step 102: training loss: 72867.82735960021
Epoch 1 step 103: training accuarcy: 0.5502
Epoch 1 step 103: training loss: 72127.23966558617
Epoch 1 step 104: training accuarcy: 0.5494
Epoch 1 step 104: training loss: 71495.21

 40%|████████████████████████████████████████████████████████████████████▊                                                                                                       | 2/5 [11:58<17:58, 359.61s/it]

...

 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                    | 3/5 [17:57<11:58, 359.37s/it]

Epoch: 3
Epoch 3 step 282: training loss: 38637.717854077724
Epoch 3 step 283: training accuarcy: 0.6291
Epoch 3 step 283: training loss: 38552.230046821634
Epoch 3 step 284: training accuarcy: 0.622
Epoch 3 step 284: training loss: 38336.60904331413
Epoch 3 step 285: training accuarcy: 0.6309
Epoch 3 step 285: training loss: 38657.76791737297
Epoch 3 step 286: training accuarcy: 0.6223000000000001
Epoch 3 step 286: training loss: 38253.28101743176
Epoch 3 step 287: training accuarcy: 0.6413
Epoch 3 step 287: training loss: 38127.59803212425
Epoch 3 step 288: training accuarcy: 0.6324000000000001
Epoch 3 step 288: training loss: 38113.81937341852
Epoch 3 step 289: training accuarcy: 0.6301
Epoch 3 step 289: training loss: 37936.57647234092
Epoch 3 step 290: training accuarcy: 0.6336
Epoch 3 step 290: training loss: 37468.8172099963
Epoch 3 step 291: training accuarcy: 0.648
Epoch 3 step 291: training loss: 38021.21492059799
Epoch 3 step 292: training accuarcy: 0.6381
Epoch 3 step 292: 

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                  | 4/5 [24:01<06:00, 360.72s/it]

Epoch: 4
Epoch 4 step 376: training loss: 33526.23385123796
Epoch 4 step 377: training accuarcy: 0.6645
Epoch 4 step 377: training loss: 33639.330065871894
Epoch 4 step 378: training accuarcy: 0.6622
Epoch 4 step 378: training loss: 33963.02187517448
Epoch 4 step 379: training accuarcy: 0.6538
Epoch 4 step 379: training loss: 33445.57665359376
Epoch 4 step 380: training accuarcy: 0.6677000000000001
Epoch 4 step 380: training loss: 33515.68651588493
Epoch 4 step 381: training accuarcy: 0.6643
Epoch 4 step 381: training loss: 33292.37318077429
Epoch 4 step 382: training accuarcy: 0.6654
Epoch 4 step 382: training loss: 33448.680799032
Epoch 4 step 383: training accuarcy: 0.6557000000000001
Epoch 4 step 383: training loss: 33370.40560762737
Epoch 4 step 384: training accuarcy: 0.665
Epoch 4 step 384: training loss: 33217.80453266455
Epoch 4 step 385: training accuarcy: 0.6665
Epoch 4 step 385: training loss: 33513.690720693725
Epoch 4 step 386: training accuarcy: 0.6518
Epoch 4 step 386: 

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [30:03<00:00, 361.14s/it]


In [26]:
del prme_model
T.cuda.empty_cache()

### Train Trans FM Model

In [27]:
trans_model = TorchTransFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
trans_model

TorchTransFM()

In [28]:
adam_opt = optim.Adam(trans_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [29]:
trans_learner = FMLearner(trans_model, adam_opt, schedular, db)
trans_learner

In [30]:
trans_learner.compile(train_col='base',
                      valid_col='base',
                      test_col='base',
                      loss_callback=trans_loss_callback)

In [18]:
trans_learner.compile(train_col='seq',
                      valid_col='seq',
                      test_col='seq',
                      loss_callback=trans_loss_callback)

In [25]:
trans_learner.compile(train_col='seq',
                      valid_col='seq',
                      test_col='seq',
                      loss_callback=trans_weight_loss_callback)

In [31]:
trans_learner.fit(epoch=3, log_dir=get_log_dir('stackoverflow', 'trans'))

  0%|                                                                                                                                                           | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 166100.56455670064
Epoch 0 step 1: training accuarcy: 0.5225
Epoch 0 step 1: training loss: 162125.3159344153
Epoch 0 step 2: training accuarcy: 0.4935
Epoch 0 step 2: training loss: 156981.66047381246
Epoch 0 step 3: training accuarcy: 0.508
Epoch 0 step 3: training loss: 152507.64515174332
Epoch 0 step 4: training accuarcy: 0.5065000000000001
Epoch 0 step 4: training loss: 147873.4152669062
Epoch 0 step 5: training accuarcy: 0.502
Epoch 0 step 5: training loss: 144006.79557214794
Epoch 0 step 6: training accuarcy: 0.507
Epoch 0 step 6: training loss: 139672.77819036233
Epoch 0 step 7: training accuarcy: 0.4985
Epoch 0 step 7: training loss: 135451.19724583233
Epoch 0 step 8: training accuarcy: 0.5095000000000001
Epoch 0 step 8: training loss: 131953.52403883662
Epoch 0 step 9: training accuarcy: 0.4895
Epoch 0 step 9: training loss: 127317.12680724992
Epoch 0 step 10: training accuarcy: 0.52
Epoch 0 step 10: training loss: 123711.71026672916
Ep

 33%|█████████████████████████████████████████████████                                                                                                  | 1/3 [01:07<02:14, 67.09s/it]

Epoch: 1
Epoch 1 step 94: training loss: 8082.237662092513
Epoch 1 step 95: training accuarcy: 0.6305000000000001
Epoch 1 step 95: training loss: 7840.816298828126
Epoch 1 step 96: training accuarcy: 0.627
Epoch 1 step 96: training loss: 7770.8700177959
Epoch 1 step 97: training accuarcy: 0.6155
Epoch 1 step 97: training loss: 7515.050589645296
Epoch 1 step 98: training accuarcy: 0.617
Epoch 1 step 98: training loss: 7174.937331224815
Epoch 1 step 99: training accuarcy: 0.6205
Epoch 1 step 99: training loss: 7118.301479377154
Epoch 1 step 100: training accuarcy: 0.617
Epoch 1 step 100: training loss: 6941.100868413972
Epoch 1 step 101: training accuarcy: 0.595
Epoch 1 step 101: training loss: 6727.702324863933
Epoch 1 step 102: training accuarcy: 0.628
Epoch 1 step 102: training loss: 6497.523892687526
Epoch 1 step 103: training accuarcy: 0.6365000000000001
Epoch 1 step 103: training loss: 6317.088705298793
Epoch 1 step 104: training accuarcy: 0.632
Epoch 1 step 104: training loss: 613

 67%|██████████████████████████████████████████████████████████████████████████████████████████████████                                                 | 2/3 [02:17<01:07, 67.97s/it]

Epoch: 2
Epoch 2 step 188: training loss: 1675.6927014041364
Epoch 2 step 189: training accuarcy: 0.7695
Epoch 2 step 189: training loss: 1643.0939907684453
Epoch 2 step 190: training accuarcy: 0.786
Epoch 2 step 190: training loss: 1600.936024236352
Epoch 2 step 191: training accuarcy: 0.801
Epoch 2 step 191: training loss: 1585.9272292212459
Epoch 2 step 192: training accuarcy: 0.8015
Epoch 2 step 192: training loss: 1662.6305064089743
Epoch 2 step 193: training accuarcy: 0.783
Epoch 2 step 193: training loss: 1598.6274747801185
Epoch 2 step 194: training accuarcy: 0.7875
Epoch 2 step 194: training loss: 1629.6449621005884
Epoch 2 step 195: training accuarcy: 0.787
Epoch 2 step 195: training loss: 1564.9140506468525
Epoch 2 step 196: training accuarcy: 0.8035
Epoch 2 step 196: training loss: 1569.20514079011
Epoch 2 step 197: training accuarcy: 0.7975
Epoch 2 step 197: training loss: 1609.5383986429197
Epoch 2 step 198: training accuarcy: 0.7855
Epoch 2 step 198: training loss: 1537.

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [03:27<00:00, 69.12s/it]


In [26]:
trans_learner.fit(epoch=3, log_dir=get_log_dir('weight_stackoverflow', 'trans'))

  0%|                                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 380319.47714286885
Epoch 0 step 1: training accuarcy: 0.5026
Epoch 0 step 1: training loss: 373132.23724375525
Epoch 0 step 2: training accuarcy: 0.5061
Epoch 0 step 2: training loss: 360396.52939689904
Epoch 0 step 3: training accuarcy: 0.511
Epoch 0 step 3: training loss: 355579.6767637768
Epoch 0 step 4: training accuarcy: 0.5022
Epoch 0 step 4: training loss: 347507.5333700695
Epoch 0 step 5: training accuarcy: 0.5041
Epoch 0 step 5: training loss: 343367.0022935191
Epoch 0 step 6: training accuarcy: 0.4963
Epoch 0 step 6: training loss: 326247.5580075149
Epoch 0 step 7: training accuarcy: 0.5058
Epoch 0 step 7: training loss: 322150.96621275716
Epoch 0 step 8: training accuarcy: 0.5049
Epoch 0 step 8: training loss: 323511.04782446893
Epoch 0 step 9: training accuarcy: 0.49710000000000004
Epoch 0 step 9: training loss: 313439.86593161046
Epoch 0 step 10: training accuarcy: 0.5067
Epoch 0 step 10: training loss: 313619.7755682239
Epoch 0 step

 33%|█████████████████████████████████████████████████████████▎                                                                                                                  | 1/3 [06:15<12:30, 375.33s/it]

Epoch: 1
Epoch 1 step 94: training loss: 66802.1171418187
Epoch 1 step 95: training accuarcy: 0.625
Epoch 1 step 95: training loss: 67588.49644747241
Epoch 1 step 96: training accuarcy: 0.6058
Epoch 1 step 96: training loss: 66497.89670721022
Epoch 1 step 97: training accuarcy: 0.6121
Epoch 1 step 97: training loss: 65625.72290045272
Epoch 1 step 98: training accuarcy: 0.6206
Epoch 1 step 98: training loss: 65535.71005794732
Epoch 1 step 99: training accuarcy: 0.6101
Epoch 1 step 99: training loss: 66916.00657243043
Epoch 1 step 100: training accuarcy: 0.6035
Epoch 1 step 100: training loss: 64045.2968736101
Epoch 1 step 101: training accuarcy: 0.627
Epoch 1 step 101: training loss: 65401.289209859286
Epoch 1 step 102: training accuarcy: 0.6137
Epoch 1 step 102: training loss: 63769.226459452315
Epoch 1 step 103: training accuarcy: 0.6164000000000001
Epoch 1 step 103: training loss: 63355.95084441677
Epoch 1 step 104: training accuarcy: 0.6135
Epoch 1 step 104: training loss: 62986.549

 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                         | 2/3 [12:30<06:15, 375.16s/it]

Epoch: 2
Epoch 2 step 188: training loss: 40578.7513814848
Epoch 2 step 189: training accuarcy: 0.6964
Epoch 2 step 189: training loss: 39215.95197827903
Epoch 2 step 190: training accuarcy: 0.7144
Epoch 2 step 190: training loss: 39799.20916198085
Epoch 2 step 191: training accuarcy: 0.7096
Epoch 2 step 191: training loss: 38954.12915400639
Epoch 2 step 192: training accuarcy: 0.7155
Epoch 2 step 192: training loss: 39733.68017931729
Epoch 2 step 193: training accuarcy: 0.6993
Epoch 2 step 193: training loss: 38904.09056988322
Epoch 2 step 194: training accuarcy: 0.7073
Epoch 2 step 194: training loss: 40139.82451927263
Epoch 2 step 195: training accuarcy: 0.6978000000000001
Epoch 2 step 195: training loss: 39913.96663611123
Epoch 2 step 196: training accuarcy: 0.7014
Epoch 2 step 196: training loss: 38856.24003989053
Epoch 2 step 197: training accuarcy: 0.7097
Epoch 2 step 197: training loss: 38769.797513904625
Epoch 2 step 198: training accuarcy: 0.7094
Epoch 2 step 198: training lo

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [18:44<00:00, 374.82s/it]


In [32]:
del trans_model
T.cuda.empty_cache()

### Train Trans Prob Model

In [14]:
feat_dim

369860

In [15]:
prob_model = TransProbFM(feature_dim=feat_dim,
                         num_dim=NUM_DIM,
                         init_mean=INIT_MEAN,
                         init_scale=INIT_SCALE)
prob_model

TransProbFM()

In [16]:
adam_opt = optim.Adam(prob_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [17]:
prob_learner = FMLearner(prob_model, adam_opt, schedular, db)
prob_learner

In [18]:
prob_learner.compile(train_col='base',
                     valid_col='base',
                     test_col='base',
                     loss_callback=trans_loss_callback)

In [19]:
prob_learner.fit(epoch=4,
                  log_dir=get_log_dir('stackoverflow', 'prob'))

  0%|                                                                                                                                                           | 0/4 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 155341.71623870675
Epoch 0 step 1: training accuarcy: 1.0


  0%|                                                                                                                                                           | 0/4 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 350.00 MiB (GPU 0; 11.00 GiB total capacity; 8.56 GiB already allocated; 94.59 MiB free; 1.99 MiB cached)

In [19]:
del prob_model
T.cuda.empty_cache()